## 1.Colab 환경 설정

In [ ]:
!pip install transformers

     |████████████████████████████████| 3.4 MB 5.4 MB/s 
     |████████████████████████████████| 3.3 MB 40.7 MB/s 
     |████████████████████████████████| 895 kB 51.5 MB/s 
     |████████████████████████████████| 61 kB 505 kB/s 
     |████████████████████████████████| 596 kB 50.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import tensorflow as tf
import torch

from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import random
import time
import datetime

In [ ]:
# GPU 확인하기
n_devices = torch.cuda.device_count()
print(n_devices)

for i in range(n_devices):
    print(torch.cuda.get_device_name(i))

1
Tesla K80


## 2.데이터셋 불러오기

In [ ]:
# #구글드라이브 연동
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
 %cd /content/drive/MyDrive/논문/뉴스모델링

/content/drive/MyDrive/논문/뉴스모델링


In [ ]:
import csv
import pandas as pd
chatbot_data = pd.read_csv('data/1222_esg.csv',encoding="utf-8")

In [ ]:
chatbot_data.shape

(17780, 4)

In [ ]:
chatbot_data.sample(n=5)

,Unnamed: 0,제목,label,text
14095,14484,"동서식품, 재난재해 대비 긴급구호키트 제작 5000만원 지원",0,"[이데일리 이성웅 기자] 동서식품은 장마, 태풍 등 여름철 재난재해에 대비한 이재민..."
10764,11045,더 오르기 전에 알바비 탈탈 털어 명품 지르는 20대,0,[머니투데이 오정은 기자] [편집자주] 이태원발 코로나19(COVID-19) 재확산...
17589,18135,[기고] 건강보험료 올리자,0,"©게티이미지뱅크 국민건강보험, 우리나라에서 특별한 제도이다. 보통 모든 국민을 ..."
5591,5726,"KT, 공적 마스크 판매 약국에 '유선 통화연결음' 무료제공",0,전국 2만3000개 약국에 '링고비즈플러스' 3개월 무료 제공 마스크 입고시간 및 ...
16200,16674,첫 ESG채권 발행한 포스코건설…'기업시민' 실천 선봉장,1,1200억원 규모 소셜본드 발행…재무건전성·사업포트폴리로 높이 평가받아 [아이뉴스2...


In [ ]:
ml=[]
for a in chatbot_data.label:
  if a == '1':
    a = 1
    a = int(a)
  else:
    a = 0
    a = int(a)
  ml.append(a)

In [ ]:
len(ml)

17780

In [ ]:
ml[0]

0

In [ ]:
chatbot_data['label'] = ml

In [ ]:
chatbot_data_shuffled = chatbot_data.sample(frac=1).reset_index(drop=True)

In [ ]:
#train data & test data 로드 
train = chatbot_data_shuffled[:16000]
test = chatbot_data_shuffled[16000:]

print(train.shape)
print(test.shape)

(16000, 4)
(1780, 4)


In [ ]:
display(train.head())
display(test.head())

,Unnamed: 0,제목,label,text
0,12838,사람 목숨을 '벌금'으로 헤아릴 수 있나?,0,중대재해기업처벌법 제정 촉구 시민사회 연속 기고 ① [이정호 민주노총 세종충남본부...
1,2498,"ESG, 미래투자 필수 아이콘이 되다",1,투자결정시 기업 비재무 건전성 평가…이미 세계적 추세로 글로벌 시장규모 30조달러 ...
2,15541,"한샘, 전국 소방관 근무·주거환경 개선 나선다",1,한샘 임직원과 용산소방서 소방관들이 기념 촬영을 하고 있다. 한샘 이주영 대외협력실...
3,4798,[더뉴스-더인터뷰] 신규 확진자 증가세 주춤...코로나19 정점 지났나?,0,NaN
4,8059,"신한은행, 2020년 수시채용 실시",0,신한은행(행장 진옥동)은 수시채용을 실시한다고 13일 밝혔다. 코로나19 여파로 취...


,Unnamed: 0,제목,label,text
16000,10196,"삼성 준법위, 이재용 사과 '절반' 인정 ""의미있지만 개선방안 필요""(종합)",0,"6일 이 부회장 '대국민 사과'에 하루 지난 7일 입장 발표 ""직접 사과, 준법 의..."
16001,8203,7주째 매일 대구 의료진에 단팥빵…조금이라도 세상 밝아지기를,0,김경미 Grit918 베이커리 대표 매출 급감에도 휴업 대신 기부 7주째 매일 대...
16002,14246,"갑을상생·소비자보호·독과점예방…공정위, `디지털 공정경제` 추진",0,공정거래위원회가 온라인 플랫폼의 입점업체 대상 갑질을 막기 위해 관련 법안을 마련한...
16003,1527,"조산구 위홈 대표 ""한국판 에어비앤비 이제 현실이 됩니다""",0,조산구 위홈 대표./사진=김정훈 기자 “큰 고비를 넘겼습니다. 이제 새로운 도약만 ...
16004,16510,9년간 수족관 생활에 지쳤나... 여수 상징 '벨루가' 죽어,0,"20일 새벽 벨루가 한 마리 폐사 아쿠아리움 측 ""폐사 원인 규명"" 동물단체 ""남은..."


## 3.Train set 전처리

In [ ]:
# CLS, SEP 붙이기 (문장의 시작, 끝)
sentences = ["[CLS] " + str(s) + " [SEP]" for s in train.제목]

In [ ]:
sentences[:5]

["[CLS] 사람 목숨을 '벌금'으로 헤아릴 수 있나? [SEP]",
 '[CLS] ESG, 미래투자 필수 아이콘이 되다 [SEP]',
 '[CLS] 한샘, 전국 소방관 근무·주거환경 개선 나선다 [SEP]',
 '[CLS] [더뉴스-더인터뷰] 신규 확진자 증가세 주춤...코로나19 정점 지났나? [SEP]',
 '[CLS] 신한은행, 2020년 수시채용 실시 [SEP]']

In [ ]:
labels = train['label'].values
labels

array([0, 1, 1, ..., 1, 0, 0])

In [ ]:
import pandas as pd
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased", do_lower_case=False)
result = tokenizer.tokenize('안녕하세요!')
print(result)

Downloading:   0%|          | 0.00/972k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.87M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

['안', '##녕', '##하', '##세', '##요', '!']


In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)
tokenized_texts = [tokenizer.tokenize(s) for s in sentences]

In [ ]:
print(sentences[0])  #토크나이징 전
print(tokenized_texts[0]) #토크나이징 후

[CLS] 사람 목숨을 '벌금'으로 헤아릴 수 있나? [SEP]
['[CLS]', '사', '##람', '목', '##숨', '##을', "'", '벌', '##금', "'", '으로', '헤', '##아', '##릴', '수', '있', '##나', '?', '[SEP]']


In [ ]:
MAX_LEN = 128 #최대 시퀀스 길이 설정
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

In [ ]:
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

In [ ]:
input_ids[0]

array([   101,   9405,  61250,   9284, 119065,  10622,    112,   9339,
        40032,    112,  29805,   9973,  16985,  85836,   9460,   9647,
        16439,    136,    102,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
      

In [ ]:
attention_masks = []

for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

In [ ]:
print(attention_masks[0])

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [ ]:
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids,
                                                                                    labels, 
                                                                                    random_state=2000, 
                                                                                    test_size=0.1)

In [ ]:
train_masks, validation_masks, _, _ = train_test_split(attention_masks, 
                                                       input_ids,
                                                       random_state=2000, 
                                                       test_size=0.1)

In [ ]:
train_inputs = torch.tensor(train_inputs)
train_labels = torch.tensor(train_labels)
train_masks = torch.tensor(train_masks)
validation_inputs = torch.tensor(validation_inputs)
validation_labels = torch.tensor(validation_labels)
validation_masks = torch.tensor(validation_masks)

In [ ]:
batch_size = 32

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

## 4.Test set 전처리

In [ ]:
# [CLS] + 문장 + [SEP]
sentences = ["[CLS] " + str(sentence) + " [SEP]" for sentence in test.제목]

In [ ]:
len(sentences)

1780

In [ ]:
labels = test['label'].values
labels

array([0, 0, 0, ..., 0, 0, 1])

In [ ]:
len(labels)

1780

In [ ]:
# [CLS] + 문장 + [SEP]
# sentences = ["[CLS] " + str(sentence) + " [SEP]" for sentence in sentences]

# 라벨 데이터
# labels = test['label'].values


# Word 토크나이저 토큰화
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

# 시퀀스 설정 및 정수 인덱스 변환 & 패딩
MAX_LEN = 128
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

# 어텐션 마스크
attention_masks = []
for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)
    
# 파이토치 텐서로 변환
test_inputs = torch.tensor(input_ids)
test_labels = torch.tensor(labels)
test_masks = torch.tensor(attention_masks)

# 배치 사이즈 설정 및 데이터 설정
batch_size = 32
test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_sampler = RandomSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

## 5.모델 생성

In [ ]:
if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

There are 1 GPU(s) available.
We will use the GPU: Tesla K80


In [ ]:
model = BertForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=2)
model.cuda()

Downloading:   0%|          | 0.00/681M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [ ]:
# 옵티마이저
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # 학습률(learning rate)
                  eps = 1e-8 
                )

# 에폭수
epochs = 7

# 총 훈련 스텝 : 배치반복 횟수 * 에폭
total_steps = len(train_dataloader) * epochs

# 스케줄러 생성
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)


## 6.모델 학습

In [ ]:
torch.cuda.empty_cache()

In [ ]:
# 정확도 계산 함수
def flat_accuracy(preds, labels):
    
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    return np.sum(pred_flat == labels_flat) / len(labels_flat)
    
    
# 시간 표시 함수
def format_time(elapsed):

    # 반올림
    elapsed_rounded = int(round((elapsed)))
    
    # hh:mm:ss으로 형태 변경
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [ ]:
labels

array([0, 0, 0, ..., 0, 0, 1])

In [ ]:
    import torch
    torch.cuda.empty_cache()
    

In [ ]:
torch.cuda.memory_summary(device=None, abbreviated=False)

'|===========================================================================|\n|                  PyTorch CUDA memory summary, device ID 0                 |\n|---------------------------------------------------------------------------|\n|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |\n|===========================================================================|\n|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |\n|---------------------------------------------------------------------------|\n| Allocated memory      |  695025 KB |  695025 KB |  695025 KB |       0 B  |\n|       from large pool |  694528 KB |  694528 KB |  694528 KB |       0 B  |\n|       from small pool |     497 KB |     497 KB |     497 KB |       0 B  |\n|---------------------------------------------------------------------------|\n| Active memory         |  695025 KB |  695025 KB |  695025 KB |       0 B  |\n|       from large pool |  694528 KB |  694528 KB |

In [ ]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
#랜덤시드 고정
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

#그래디언트 초기화
model.zero_grad()

# 학습
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # 시작 시간 설정
    t0 = time.time()

    # 로스 초기화
    total_loss = 0

    # 훈련모드로 변경
    model.train()
        
    # 데이터로더에서 배치만큼 반복하여 가져옴
    for step, batch in enumerate(train_dataloader):
        # 경과 정보 표시
        if step % 500 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch

        # Forward 수행                
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask, 
                        labels=b_labels)
        
        # 로스 구함
        loss = outputs[0]

        # 총 로스 계산
        total_loss += loss.item()

        # Backward 수행으로 그래디언트 계산
        loss.backward()

        # 그래디언트 클리핑
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # 그래디언트를 통해 가중치 파라미터 업데이트
        optimizer.step()

        # 스케줄러로 학습률 감소
        scheduler.step()

        # 그래디언트 초기화
        model.zero_grad()

    # 평균 로스 계산
    avg_train_loss = total_loss / len(train_dataloader)            

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))
        
    # ========================================
    #               Validation
    # ========================================

    print("")
    print("Running Validation...")

    #시작 시간 설정
    t0 = time.time()

    # 평가모드로 변경
    model.eval()

    # 변수 초기화
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    # 데이터로더에서 배치만큼 반복하여 가져옴
    for batch in validation_dataloader:
        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch
        
        # 그래디언트 계산 안함
        with torch.no_grad():     
            # Forward 수행
            outputs = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)
        
        # 로스 구함
        logits = outputs[0]

        # CPU로 데이터 이동
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        # 출력 로짓과 라벨을 비교하여 정확도 계산
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1

    print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))

print("")
print("Training complete!")


======== Epoch 1 / 7 ========
Training...

  Average training loss: 0.41
  Training epcoh took: 0:10:11

Running Validation...
  Accuracy: 0.88
  Validation took: 0:00:24

======== Epoch 2 / 7 ========
Training...

  Average training loss: 0.29
  Training epcoh took: 0:10:10

Running Validation...
  Accuracy: 0.88
  Validation took: 0:00:24

======== Epoch 3 / 7 ========
Training...

  Average training loss: 0.23
  Training epcoh took: 0:10:10

Running Validation...
  Accuracy: 0.89
  Validation took: 0:00:24

======== Epoch 4 / 7 ========
Training...

  Average training loss: 0.18
  Training epcoh took: 0:10:10

Running Validation...
  Accuracy: 0.89
  Validation took: 0:00:24

======== Epoch 5 / 7 ========
Training...

  Average training loss: 0.14
  Training epcoh took: 0:10:10

Running Validation...
  Accuracy: 0.89
  Validation took: 0:00:24

======== Epoch 6 / 7 ========
Training...

  Average training loss: 0.11
  Training epcoh took: 0:10:10

Running Validation...
  Accuracy: 

## 7.테스트셋 평가

In [ ]:
#시작 시간 설정
t0 = time.time()

# 평가모드로 변경
model.eval()

# 변수 초기화
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0

# 데이터로더에서 배치만큼 반복하여 가져옴
for step, batch in enumerate(test_dataloader):
    # 경과 정보 표시
    if step % 100 == 0 and not step == 0:
        elapsed = format_time(time.time() - t0)
        print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(test_dataloader), elapsed))

    # 배치를 GPU에 넣음
    batch = tuple(t.to(device) for t in batch)
    
    # 배치에서 데이터 추출
    b_input_ids, b_input_mask, b_labels = batch
    
    # 그래디언트 계산 안함
    with torch.no_grad():     
        # Forward 수행
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)
    
    # 로스 구함
    logits = outputs[0]

    # CPU로 데이터 이동
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()
    
    # 출력 로짓과 라벨을 비교하여 정확도 계산
    tmp_eval_accuracy = flat_accuracy(logits, label_ids)
    eval_accuracy += tmp_eval_accuracy
    nb_eval_steps += 1

print("")
print("Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
print("Test took: {:}".format(format_time(time.time() - t0)))


Accuracy: 0.87
Test took: 0:00:27


## 8.새로운 문장 테스트

In [ ]:
# 입력 데이터 변환
def convert_input_data(sentences):

    # BERT의 토크나이저로 문장을 토큰으로 분리
    tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

    # 입력 토큰의 최대 시퀀스 길이
    MAX_LEN = 128

    # 토큰을 숫자 인덱스로 변환
    input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
    
    # 문장을 MAX_LEN 길이에 맞게 자르고, 모자란 부분을 패딩 0으로 채움
    input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

    # 어텐션 마스크 초기화
    attention_masks = []

    # 어텐션 마스크를 패딩이 아니면 1, 패딩이면 0으로 설정
    # 패딩 부분은 BERT 모델에서 어텐션을 수행하지 않아 속도 향상
    for seq in input_ids:
        seq_mask = [float(i>0) for i in seq]
        attention_masks.append(seq_mask)

    # 데이터를 파이토치의 텐서로 변환
    inputs = torch.tensor(input_ids)
    masks = torch.tensor(attention_masks)

    return inputs, masks

In [ ]:
# 문장 테스트
def test_sentences(sentences):

    # 평가모드로 변경
    model.eval()

    # 문장을 입력 데이터로 변환
    inputs, masks = convert_input_data(sentences)

    # 데이터를 GPU에 넣음
    b_input_ids = inputs.to(device)
    b_input_mask = masks.to(device)
            
    # 그래디언트 계산 안함
    with torch.no_grad():     
        # Forward 수행
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)

    # 로스 구함
    logits = outputs[0]

    # CPU로 데이터 이동
    logits = logits.detach().cpu().numpy()

    return logits

In [ ]:
logits = test_sentences(['BNK금융, 소외계층 설 명절 위해 6억5000만원 지원'])
print(logits)

if np.argmax(logits) == 1 :
    print("ESG관련 기사")
elif np.argmax(logits) == 0 :
    print("esg관련 기사 아님")

[[-2.20816    2.3684204]]
ESG관련 기사


In [ ]:
logits = test_sentences(['삼성증권, 소외계층 설 명절 위해 6억5000만원 기부'])

print(logits)
if np.argmax(logits) == 1 :
    print("ESG관련 기사")
elif np.argmax(logits) == 0 :
    print("일반 기사")

[[-2.1294107  2.2796178]]
ESG관련 기사


In [ ]:
logits = test_sentences(['친환경적 비건 식품을 위한 금액'])

print(logits)
if np.argmax(logits) == 1 :
    print("ESG관련 기사")
elif np.argmax(logits) == 0 :
    print("일반 기사")

[[-3.1546345  3.3695867]]
ESG관련 기사


In [ ]:
logits = test_sentences(['강인규 나주시장 "에너지신산업 거점 자리매김"'])

print(logits)
if np.argmax(logits) == 1 :
    print("ESG관련 기사")
elif np.argmax(logits) == 0 :
    print("일반 기사")

[[ 3.5309596 -3.6520562]]
일반 기사


In [ ]:
logits = test_sentences(['바디프랜드, 침수 피해 안마의자 무상 교체'])

print(logits)
if np.argmax(logits) == 1 :
    print("ESG관련 기사")
elif np.argmax(logits) == 0 :
    print("일반 기사")

[[-2.6705334  2.7503622]]
ESG관련 기사


In [ ]:
logits = test_sentences(['13년 투병 아내 이제 하늘로…가습기살균제 사망 1559명'])

print(logits)
if np.argmax(logits) == 1 :
    print("ESG관련 기사")
elif np.argmax(logits) == 0 :
    print("일반 기사")

[[-1.3984739  1.719834 ]]
ESG관련 기사


In [ ]:
logits = test_sentences(['러시아서 세계 첫 코로나 백신 등록…안전성 우려 外'])

print(logits)
if np.argmax(logits) == 1 :
    print("ESG관련 기사")
elif np.argmax(logits) == 0 :
    print("일반 기사")

[[ 3.5739849 -3.7667382]]
일반 기사


In [ ]:
logits = test_sentences(['“함께가요 미래로!” 세계 청소년들을 향한 삼성전자의 따뜻한 외침'])

print(logits)
if np.argmax(logits) == 1 :
    print("ESG관련 기사")
elif np.argmax(logits) == 0 :
    print("일반 기사")

[[-2.9305823  3.078593 ]]
ESG관련 기사


In [ ]:

logits = test_sentences(['산재 피해자에게 지속되는 고통…기업 처벌법 마련해야'])

print(logits)
if np.argmax(logits) == 1 :
    print("ESG관련 기사")
elif np.argmax(logits) == 0 :
    print("일반 기사")

[[-1.5107225  1.7615894]]
ESG관련 기사


In [ ]:

logits = test_sentences(['신한카드, 푸조와 ‘Green Mobility’ 캠페인 실시'])

print(logits)
if np.argmax(logits) == 1 :
    print("ESG관련 기사")
elif np.argmax(logits) == 0 :
    print("일반 기사")

[[-0.62156206  0.5463852 ]]
ESG관련 기사


In [ ]:
logits = test_sentences(['기안84의 끝없는 여성혐오 생산, 그리고 방관자들'])

print(logits)
if np.argmax(logits) == 1 :
    print("ESG관련 기사")
elif np.argmax(logits) == 0 :
    print("일반 기사")

[[ 3.316391  -3.4005802]]
일반 기사


In [ ]:
logits = test_sentences(['애플 2천조, 네이버 45조, 카카오 24조..플랫폼 어찌해야 하나'])

print(logits)
if np.argmax(logits) == 1 :
    print("ESG관련 기사")
elif np.argmax(logits) == 0 :
    print("일반 기사")

[[ 3.0986617 -3.072463 ]]
일반 기사


In [ ]:
logits = test_sentences(['국내 소비자 95% "환경오염 심각" 한국피앤지 설문조사 발표'])

print(logits)
if np.argmax(logits) == 1 :
    print("ESG관련 기사")
elif np.argmax(logits) == 0 :
    print("일반 기사")

[[ 3.6281345 -3.8029668]]
일반 기사


In [ ]:
logits = test_sentences(['홈코노미·멘탈데믹…소비 新트렌드 분석 서적'])

print(logits)
if np.argmax(logits) == 1 :
    print("ESG관련 기사")
elif np.argmax(logits) == 0 :
    print("일반 기사")

[[ 3.529864 -3.747797]]
일반 기사


In [ ]:


logits = test_sentences(['한솔홀딩스, 美LACP 지속가능경영보고서 ‘금상’ 수상'])

print(logits)
if np.argmax(logits) == 1 :
    print("ESG관련 기사")
elif np.argmax(logits) == 0 :
    print("일반 기사")

[[-2.4938288  2.7447867]]
ESG관련 기사
